#  赛题十五 —— 论文复现自验报告

赛题内容：利用 MindQuantum 实现量子卷积神经网络求解量子多体问题

论文题目：[An Application of Quantum Machine Learning on Quantum Correlated Systems: Quantum Convolutional Neural Network as a Classifier for Many-Body Wavefunctions from the Quantum Variational Eigensolver](https://arxiv.org/abs/2111.05076)
 
复现要求：基于 MindQuantum 实现 `图 4` 中的量子卷积神经网络，并在 N = 4、8、12 的情况下实现对顺磁性和铁磁性的分类，精度要求达到 90% 以上。

## 赛题介绍

基于量子力学的量子卷积神经网络可以用来代替经典神经网络来完成分类问题。复现论文中，提出了采用量子卷积神经网络来预测某一维横场伊辛模型的基态是处于顺磁相还是铁磁相。
该方案可以用于确定量子临界点 —— 这对于研究量子多体系统的相变问题具有重要意义。

一维横场伊辛模型的哈密顿量为：
$$H=-J\Sigma_{i=1}^{N}\sigma_i^z\sigma_{i+1}^z-\Gamma\Sigma_{i=1}^N\sigma_i^z.$$
其中，$\sigma_i^\alpha$ 为作用在第 $i$ 个比特上的泡利算符。$J$ 为最近邻自旋的耦合强度，此处设 $J=1$ 并将其作为系统的能量单位。系统满足周期性边界条件。当 $\Gamma>J$ 时，系统的基态处于顺磁相，而当 $\Gamma <J$ 时，系统的基态处于铁磁相。本项目就将训练量子卷积神经网络，预测不同 $\Gamma$ 值下的系统基态，是处于顺磁相还是处于铁磁相。

## 数据集

文章原本所用数据集为 `Tensorflow_quantum` dataset 下的 `tfi_chain` 数据集。该数据集共包括 81 个数据点，每个数据点包括 一维横场伊辛模型中的参数 $\Gamma\in[0.2,1.8]$，对应的构造基态的编码线路参数、分类标签（基态处于何种相，1 个参数，铁磁相表示为 `0`， 而顺磁相表示为 `1`）等。

**数据集的改进：**

**由于原论文所用数据集较小，不足以同时满足训练和测试之用，原论文作者采用了插值的方法生成更多的样本点以准确反映训练效果。在本项目中，我们则采用和 `Tensorflow_quantum` 数据集相同的生成方式，生成更多的样本点，这种数值增强方法在可靠性上要优于原论文的插值法**。

**为训练和验证模型效果，对 4， 8， 和 12 个量子比特系统的情况，我们都分别将一维横场伊辛模型中的参数 $\Gamma\in[0.0, 2.0]$ 以 `0.004` 为间隔，均匀采样 500 个点构成数据集。对每个样本点，我们都以对应的参数 $\Gamma$ 构建哈密顿量，训练拟设线路求解基态，直到收敛（相邻两次训练得到的损失函数相差小于 1E-10）。对绝大多数点，都可以在 100 - 500 次迭代后达到收敛效果，但在 12 量子比特情况下，有 3 个点在超过 2000 次训练后仍未收敛，我们对其做了抛弃处理。所得的线路参数 `x` 以矢量形式 与对应标签 `y` （`0` 代表铁磁相，`1` 代表顺磁相）一同构成总数据集，之后随机打散并拆分，其中 300 个组成训练集，100 个组成验证集，100 个组成测试集。**

## 编码线路

待分类的系统基态由编码线路制备得到。下面我们以 4 比特情况为例，展示具有一层 `Hadmard 变换` 和 两层 `基本结构` 的基态编码线路：

In [96]:
from mindquantum import *

def ham_layer(n_qubits=4, prefix='0'):
    circ_ = Circuit([ZZ(f'{0}').on([i, i+1]) for i in range(n_qubits-1)])
    circ_ += ZZ(f'{0}').on([n_qubits-1, 0])
    circ_ += Circuit([RX(f'{1}').on(i) for i in range(n_qubits)])
    circ_ += BarrierGate()
    circ_ = add_prefix(circ_, prefix)
    return circ_

def h_layer(n_qubits):
    circ_ = Circuit([H.on(i) for i in range(n_qubits)])
    circ_ += BarrierGate()
    return circ_

n_qubits = 4

encoder = h_layer(n_qubits)
encoder += ham_layer(n_qubits, prefix='0')
encoder += ham_layer(n_qubits, prefix='1')
encoder.svg()

该线路共包含 2 层 `基本结构`， 4 个参数（对于 8 和 12 比特情况，则包含 4 层 8 个参数 和 10 层 20 个参数）。该线路参数和标签都分别保存在数据集 `4/8/12_qubit_data_x_set` 和 `4/8/12_qubit_data_y_set` 中。

量子卷积线路通过施加卷积操作，将卷积核所作用的量子比特纠缠起来，提取信息。在作为分类器的拟设结构上，我们采用量子卷积神经网络。**但相比原论文方案，我们做了一定的改进：**

**1. 改进了拟设线路结构。由于所研究的物理系统具有周期性边界条件，我们设计的拟设结构在第一卷积层也使用了周期性边界条件，这使得原始局部信息可以得到更好地提取。实验上，这种结构在环形比特阵列中也可以很容易地实现；**

**2. 更换了卷积核。我们采用的卷积核量子门数更少，为理论上的最小值，而且能达到相同的卷积效果（可以完成任意双比特操作）；**

**3. 用丢比特的方式来代替原池化模块。通过直接降维，引入非线性，不需要任何量子门，效率更高；**

**4. 原文采用以所剩最后一个量子比特的期望值作为分类依据，比如当 $\sigma_3^z$ 的期望值小于 0 则分类为 `0`，否则分类为 `1`。本项目则通过比较所剩两个比特 $\sigma^z$ 算符期望值大小，来作为分类依据。比如，若 $\sigma_3^z$ 期望值大于 $\sigma_5^z$ 的期望值，则规定该预测分类为 `0`，否则为 `1`。在我们开发的过程中，发现后者的效果更好、更稳定。**


以 8 量子比特为例，下图展示了量子线路的整体框架：

![量子线路](./src/circ.png)

其中，绿色的 Encoder 就是上面介绍的编码线路，其输出就是对应哈密顿量的基态，而由蓝色方块构成的拟设 Ansatz 线路则是对量子态进行分类的卷积神经网络。其中，每个蓝色的卷积核的结构为：

![卷积核结构](./src/conv_circ.png)

## 自验结果

通过执行 `main.ipynb` 中对应的程序，可分别得到 4、8 和 12 比特情况下的训练结果。

对于 4 比特情况，经过大约 50 次训练后，模型收敛，对验证集的分类准确度达到了 1.0。 在测试集上准确率为 1.0。 (参见 `main.ipynb` 中对应的运行结果，或 `src` 文件夹下 `result_4_qubit.png`)

![4 比特情况](./src/result_4_qubit.png)


对于 8 比特情况，经过大约 170 余次训练后，模型收敛，对验证集的分类准确度达到了 1.0。 在测试集上准确率为 1.0。 (参见 `main.ipynb` 中对应的运行结果，或 `src` 文件夹下 `result_8_qubit.png`)

![8 比特情况](./src/result_8_qubit.png )

对于 12 比特情况，经过大约 100 次训练后，模型收敛，对测试集的分类准确度达到了 1.0。（参见 `main.ipynb` 中对应的运行结果，或 `src` 文件夹下的 `result_12_qubit.png`）

![12 比特情况](./src/result_12_qubit.png )


## 训练

模型训练：

训练开始前我们随机初始拟设网络的参数。

训练过程中，我们每个 step 都从容量为 300 的训练集中随机抽取 batch_size=5 个样本点，并求出在验证集上的效果。

当验证集的准确率达到指定的高精度后（比如 1.0 ），我们即计算在测试集上的表现。

执行 `main.ipynb` 文件中对应的程序，可自动对 4、8 和 12 比特情况的模型进行训练，并得到训练结果。
